In [2]:
# S-DES Implementation in Python (Modified for brevity)

# S-Boxes
S0 = [[1, 0, 3, 2], [3, 2, 1, 0], [0, 2, 1, 3], [3, 1, 3, 2]]
S1 = [[0, 1, 2, 3], [2, 0, 1, 3], [3, 0, 1, 0], [2, 1, 0, 3]]

# Helper functions
def permute(bits, pattern): return [bits[i - 1] for i in pattern]
def left_shift(bits, n): return bits[n:] + bits[:n]
def str_to_bits(s): return [int(bit) for bit in s]
def bits_to_str(bits): return ''.join(str(bit) for bit in bits)

def s_box(bits, box):
    row, col = (bits[0] << 1) + bits[3], (bits[1] << 1) + bits[2]
    val = box[row][col]
    return [val >> 1, val & 1]

def fk(bits, subkey):
    EP = [4, 1, 2, 3, 2, 3, 4, 1]
    P4 = [2, 4, 3, 1]
    
    left, right = bits[:4], bits[4:]
    expanded = permute(right, EP)
    xor_result = [b ^ k for b, k in zip(expanded, subkey)]
    
    sbox_output = s_box(xor_result[:4], S0) + s_box(xor_result[4:], S1)
    p4 = permute(sbox_output, P4)
    
    return [l ^ p for l, p in zip(left, p4)] + right

def generate_keys(key_bits):
    P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    P8 = [6, 3, 7, 4, 8, 5, 10, 9]

    permuted = permute(key_bits, P10)
    left, right = permuted[:5], permuted[5:]
    
    # First key with 1-bit shift
    left1, right1 = left_shift(left, 1), left_shift(right, 1)
    k1 = permute(left1 + right1, P8)
    
    # Second key with 2-bit additional shift
    left2, right2 = left_shift(left1, 2), left_shift(right1, 2)
    k2 = permute(left2 + right2, P8)
    
    return k1, k2

def process(text, key, decrypt=False):
    IP = [2, 6, 3, 1, 4, 8, 5, 7]
    IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]
    
    key_bits = str_to_bits(key)
    text_bits = str_to_bits(text)
    k1, k2 = generate_keys(key_bits)
    
    # Use keys in reverse order for decryption
    if decrypt:
        k1, k2 = k2, k1
        
    permuted = permute(text_bits, IP)
    after_round1 = fk(permuted, k1)
    swapped = after_round1[4:] + after_round1[:4]  # Switch function
    after_round2 = fk(swapped, k2)
    
    return bits_to_str(permute(after_round2, IP_inv))

# Main encryption/decryption functions
def encrypt(plaintext, key): return process(plaintext, key)
def decrypt(ciphertext, key): return process(ciphertext, key, decrypt=True)

# Example
plaintext = '10101010'
key = '1010000010'

cipher = encrypt(plaintext, key)
print("Encrypted:", cipher)

decrypted = decrypt(cipher, key)
print("Decrypted:", decrypted)

Encrypted: 10001101
Decrypted: 10101010
